In [2]:
import numpy as np
import pandas as pd
from dateutil import parser as dateparser
from datetime import datetime

In [156]:
# def is_us_state(row):
#     loc = row['location_key']
#     return loc.startswith('US') and loc.count('_') == 1
# def get_state(row):
#     row.location_key = row.location_key.split('_')[1]
#     return row
# def get_us_states_df(path):
#     df = pd.read_csv(path).dropna(subset=['location_key'])
#     df = df[df.apply(is_us_state, axis=1)]
#     return df.apply(get_state, axis=1)
def is_us_state(row):
    loc = row['location_key']
    return loc.startswith('US') and loc.count('_') <= 1
def get_us_states_df(path):
    df = pd.read_csv(path).dropna(subset=['location_key'])
    return df[df.apply(is_us_state, axis=1)]

In [157]:
epidemiology = get_us_states_df('../data/open_data/epidemiology.csv')

In [165]:
epidemiology.head()

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
8884847,2020-01-01,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8884848,2020-01-02,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8884849,2020-01-03,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8884850,2020-01-04,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8884851,2020-01-05,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN


In [158]:
demo = get_us_states_df('../data/open_data/demographics.csv')

In [159]:
em_declare = get_us_states_df('../data/open_data/lawatlas-emergency-declarations.csv')

In [160]:
geography = get_us_states_df('../data/open_data/geography.csv')

In [161]:
health = get_us_states_df('../data/open_data/health.csv')

In [162]:
hospitalization = get_us_states_df('../data/open_data/hospitalizations.csv')

In [163]:
mobility = get_us_states_df('../data/open_data/mobility.csv')

In [167]:
mobility[mobility.location_key == 'US']

,date,location_key,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential
3240321,2020-02-15,US,6.0,2.0,15.0,3.0,2.0,-1.0
3240322,2020-02-16,US,7.0,1.0,16.0,2.0,0.0,-1.0
3240323,2020-02-17,US,6.0,0.0,28.0,-9.0,-24.0,5.0
3240324,2020-02-18,US,0.0,-1.0,6.0,1.0,0.0,1.0
3240325,2020-02-19,US,2.0,0.0,8.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...
3241108,2022-04-12,US,-9.0,-2.0,24.0,-24.0,-21.0,4.0
3241109,2022-04-13,US,-8.0,-3.0,18.0,-25.0,-21.0,4.0
3241110,2022-04-14,US,-5.0,1.0,24.0,-23.0,-22.0,4.0
3241111,2022-04-15,US,-6.0,0.0,27.0,-26.0,-35.0,7.0


In [355]:
def is_us_state_(row):
    loc = row['facility_sub_region_1_code']
    return loc.startswith('US') and loc.count('-') == 1
def get_state_(row):
    row.facility_sub_region_1_code = row.facility_sub_region_1_code.replace('-', '_')
    return row
def get_us_states_df_(path):
    df = pd.read_csv(path).dropna(subset=['facility_sub_region_1_code'])
    df = df[df.apply(is_us_state_, axis=1)]
    return df.apply(get_state_, axis=1)
facility = get_us_states_df_('../data/open_data/facility-boundary-us-drive.csv')
facility = facility.groupby('facility_sub_region_1_code').agg(vaccine_facility_cnt=('facility_place_id', 'count'))
facility.index.names = ['location_key']

In [357]:
facility = facility.reset_index()

In [164]:
vaccinations = get_us_states_df('../data/open_data/vaccinations.csv')

In [367]:
weather = get_us_states_df('../data/open_data/weather.csv')

In [218]:
# prepare timeseries dataset
index_key = ['location_key', 'date']

In [280]:
epi = epidemiology.dropna(subset=['date'])
epi.date = epi.date.apply(dateparser.parse)
epi = epi.groupby(['location_key', epi.date.dt.year, epi.date.dt.month]).agg(\
    new_confirmed=('new_confirmed', 'mean'),\
    new_deceased=('new_deceased', 'mean'),\
    new_recovered=('new_recovered', 'mean'),\
    new_tested=('new_tested', 'mean'),\
    cumulative_confirmed=('cumulative_confirmed', 'min'),\
    cumulative_deceased=('cumulative_deceased', 'min'),\
    cumulative_recovered=('cumulative_recovered', 'min'),\
    cumulative_tested=('cumulative_tested', 'min')\
)
epi.index = epi.index.set_names(['location_key', 'year', 'month'])
epi = epi.reset_index()
epi['date'] = epi.apply(lambda row: datetime(year=row['year'], month=row['month'], day=1), axis=1)
epi = epi.drop(columns=['year', 'month'])

In [283]:
hos = hospitalization.dropna(subset=['date'])
hos.date = hos.date.apply(dateparser.parse)
hos = hos.groupby(['location_key', hos.date.dt.year, hos.date.dt.month]).agg(\
    new_hospitalized_patients=('new_hospitalized_patients', 'mean'),\
    cumulative_hospitalized_patients=('cumulative_hospitalized_patients', 'min')
)
hos.index = hos.index.set_names(['location_key', 'year', 'month'])
hos = hos.reset_index()
hos['date'] = hos.apply(lambda row: datetime(year=row['year'], month=row['month'], day=1), axis=1)
hos = hos.drop(columns=['year', 'month'])

In [287]:
mob = mobility.dropna(subset=['date'])
mob.date = mob.date.apply(dateparser.parse)
mob = mob.groupby(['location_key', mob.date.dt.year, mob.date.dt.month]).mean()
mob.index = mob.index.set_names(['location_key', 'year', 'month'])
mob = mob.reset_index()
mob['date'] = mob.apply(lambda row: datetime(year=row['year'], month=row['month'], day=1), axis=1)
mob = mob.drop(columns=['year', 'month'])

In [303]:
vax = vaccinations.dropna(subset=['date'])
vax.date = vax.date.apply(dateparser.parse)

# get cumulative_persons columns and new_persons columns
vax_cumu_columns = []
vax_new_columns = []
for col in vax.columns:
    if col.startswith('cumulative') or col.startswith('total'):
        vax_cumu_columns.append(col)
    elif col.startswith('new'):
        vax_new_columns.append(col)

vax_cumu = vax[index_key + vax_cumu_columns]
vax_cumu = vax_cumu.groupby(['location_key', vax_cumu.date.dt.year, vax_cumu.date.dt.month]).min()
vax_cumu.index = vax_cumu.index.set_names(['location_key', 'year', 'month'])
vax_cumu = vax_cumu.reset_index()
vax_cumu['date'] = vax_cumu.apply(lambda row: datetime(year=row['year'], month=row['month'], day=1), axis=1)
vax_cumu = vax_cumu.drop(columns=['year', 'month'])

vax_new = vax[index_key + vax_new_columns]
vax_new = vax_new.groupby(['location_key', vax_new.date.dt.year, vax_new.date.dt.month]).mean()
vax_new.index = vax_new.index.set_names(['location_key', 'year', 'month'])
vax_new = vax_new.reset_index()
vax_new['date'] = vax_new.apply(lambda row: datetime(year=row['year'], month=row['month'], day=1), axis=1)
vax_new = vax_new.drop(columns=['year', 'month'])

vax_new[vax_cumu_columns] = vax_cumu[vax_cumu_columns]
vax = vax_new

In [312]:
us_timeseries_data = epi.merge(hos, how='outer', left_on=index_key, right_on=index_key)
us_timeseries_data = us_timeseries_data.merge(mob, how='outer', left_on=index_key, right_on=index_key)
us_timeseries_data = us_timeseries_data.merge(vax, how='outer', left_on=index_key, right_on=index_key)

In [388]:
us_timeseries_data.to_csv('../data/open_data/us_timeseries_data.csv', index=False)

In [315]:
# prepare per state dataset

In [329]:
vax_per_state = vax[['location_key'] + vax_cumu_columns].groupby('location_key').max().drop(columns=['total_persons_vaccinated_sinovac', 'total_persons_fully_vaccinated_sinovac', 'total_vaccine_doses_administered_sinovac']).reset_index()

In [336]:
cases_deaths_per_state = epi[['location_key', 'cumulative_confirmed', 'cumulative_deceased']].groupby('location_key').max()
cases_deaths_per_state['death_rate'] = cases_deaths_per_state.cumulative_deceased / cases_deaths_per_state.cumulative_confirmed

In [358]:
facility_cnt_per_state = facility[['location_key', 'vaccine_facility_cnt']]

In [361]:
population_per_state = demo[['location_key', 'population']]

In [363]:
geography_per_state = geography

In [365]:
life_expectancy_per_state = health[['location_key', 'life_expectancy']]

In [383]:
weather_per_state = weather.groupby('location_key')[['location_key', 'average_temperature_celsius', 'rainfall_mm', 'snowfall_mm', 'relative_humidity']].mean().reset_index()

In [386]:
tmp = cases_deaths_per_state.merge(vax_per_state, how='outer', left_on='location_key', right_on='location_key')
tmp = tmp.merge(facility_cnt_per_state, how='outer', left_on='location_key', right_on='location_key')
tmp = tmp.merge(population_per_state, how='outer', left_on='location_key', right_on='location_key')
tmp = tmp.merge(geography_per_state, how='outer', left_on='location_key', right_on='location_key')
tmp = tmp.merge(life_expectancy_per_state, how='outer', left_on='location_key', right_on='location_key')
tmp = tmp.merge(weather_per_state, how='outer', left_on='location_key', right_on='location_key')
us_per_state_data = tmp

In [389]:
us_per_state_data.to_csv('../data/open_data/us_per_state_data.csv', index=False)

In [28]:
# rename columns
# state = pd.read_csv('../data/open_data/us_per_state_data.csv')
timeseries = pd.read_csv('../data/open_data/us_timeseries_data.csv')
translation = {
    'new_confirmed': 'Average Daily Confirmed',
    'new_deceased': 'Average Daily Deceased',
    'new_recovered': 'Average Daily Recovered',
    'new_tested': 'Average Daily Tested',
    'new_hospitalized_patients': 'Average Daily Hospitalized',
    'new_persons_vaccinated': 'Average Daily Vaccinated',
    'new_persons_fully_vaccinated': 'Average Daily Fully Vaccinated',
    'new_vaccine_doses_adminstered': 'Average Daily Doses Administered'
}
timeseries.rename(columns=translation, inplace=True)
timeseries.to_csv('../data/open_data/us_timeseries_data.csv', index=False)


In [38]:
# delete timeseries future data
timeseries = pd.read_csv('../data/open_data/us_timeseries_data.csv', parse_dates=['date'])
timeseries = timeseries[timeseries.date < datetime(year=2022, month=5, day=1)]
timeseries.to_csv('../data/open_data/us_timeseries_data.csv', index=False)

In [42]:
for i in timeseries.columns:
    print(i)

location_key
Average Daily Confirmed
Average Daily Deceased
Average Daily Recovered
Average Daily Tested
cumulative_confirmed
cumulative_deceased
cumulative_recovered
cumulative_tested
date
Average Daily Hospitalized
cumulative_hospitalized_patients
mobility_retail_and_recreation
mobility_grocery_and_pharmacy
mobility_parks
mobility_transit_stations
mobility_workplaces
mobility_residential
Average Daily Vaccinated
Average Daily Fully Vaccinated
new_vaccine_doses_administered
new_persons_vaccinated_pfizer
new_persons_fully_vaccinated_pfizer
new_vaccine_doses_administered_pfizer
new_persons_vaccinated_moderna
new_persons_fully_vaccinated_moderna
new_vaccine_doses_administered_moderna
new_persons_vaccinated_janssen
new_persons_fully_vaccinated_janssen
new_vaccine_doses_administered_janssen
new_persons_vaccinated_sinovac
new_persons_fully_vaccinated_sinovac
new_vaccine_doses_administered_sinovac
cumulative_persons_vaccinated
cumulative_persons_fully_vaccinated
cumulative_vaccine_doses_admi

In [39]:
state = pd.read_csv('../data/open_data/us_per_state_data.csv')
state

,location_key,cumulative_confirmed,cumulative_deceased,death_rate,cumulative_persons_vaccinated,cumulative_persons_fully_vaccinated,cumulative_vaccine_doses_administered,cumulative_persons_vaccinated_pfizer,cumulative_persons_fully_vaccinated_pfizer,cumulative_vaccine_doses_administered_pfizer,...,longitude,elevation_m,area_sq_km,area_rural_sq_km,area_urban_sq_km,life_expectancy,average_temperature_celsius,rainfall_mm,snowfall_mm,relative_humidity
0,US,77678440.0,939599.0,0.012096,256085694.0,217703007.0,563633255.0,NaN,124885538.0,331363417.0,...,-98.579500,NaN,9831510.0,8549545.0,802053.0,78.539024,10.871091,1.399037,92.570390,60.781654
1,US_AK,246144.0,1192.0,0.004843,506738.0,451464.0,1116746.0,NaN,249701.0,639612.0,...,-150.000000,580.0,1717856.0,NaN,NaN,78.000000,-3.599809,0.997709,612.823064,68.334426
2,US_AL,1295468.0,19313.0,0.014908,3054697.0,2484952.0,6167392.0,NaN,1295289.0,3306728.0,...,-86.700000,152.0,135765.0,NaN,NaN,75.100000,17.060451,4.312946,NaN,72.358161
3,US_AR,833029.0,11237.0,0.013489,1997686.0,1629033.0,4138633.0,NaN,853775.0,2247426.0,...,-92.200000,198.0,137733.0,NaN,NaN,75.600000,15.566155,3.614283,119.878039,71.249434
4,US_AS,3756.0,0.0,0.000000,44726.0,40484.0,104022.0,NaN,30271.0,78869.0,...,-170.707500,765.0,199.0,NaN,NaN,NaN,27.280592,8.623093,NaN,84.953218
5,US_AZ,2007180.0,29268.0,0.014582,5261232.0,4435616.0,11772372.0,NaN,2469036.0,6760154.0,...,-111.656944,1250.0,295234.0,NaN,NaN,78.700000,18.074032,0.554230,156.321760,32.991649
6,US_CA,8494005.0,88207.0,0.010385,32619132.0,28108422.0,72704550.0,NaN,16390011.0,43322292.0,...,-120.000000,884.0,423970.0,NaN,NaN,80.800000,16.477900,0.523286,NaN,55.871173
7,US_CO,1352676.0,12174.0,0.009000,4540950.0,4017897.0,10452085.0,NaN,2274216.0,6065219.0,...,-105.500000,2073.0,269837.0,NaN,NaN,80.000000,6.126122,0.628572,45.495669,43.436773
8,US_CT,737276.0,10779.0,0.014620,3374089.0,2795957.0,7396401.0,NaN,1622288.0,4356303.0,...,-72.700000,152.0,14357.0,NaN,NaN,80.400000,10.742341,2.383655,102.388276,66.598796
9,US_DC,136886.0,1331.0,0.009723,683067.0,514463.0,1459617.0,NaN,299235.0,896050.0,...,-77.036667,NaN,177.0,NaN,NaN,NaN,13.248731,1.863773,41.728571,66.020083
